# Digital Elevation Model

A Digital Elevation Model (DEM) is a 3D representation of a terrain’s surface, created from terrain elevation data. It represents the earth’s surface in a raster format, where each cell or pixel holds the elevation value at that location.

Used to analyze topography, such as `slope`, `aspect`, and `curvature`, which are essential for understanding landforms and landscape features.

This chapter covers how to create, process, and analyze DEMs using Python and shell scripts. We will walk through the process of creating a GeoTIFF file for a specific region, reprojection and resampling of DEMs, and extracting various features from DEMs

## Creating a GeoTIFF Template for the Western U.S.

Our goal is to create a GeoTIFF file that serves as a template for the western U.S. This GeoTIFF will have a specified spatial extent and resolution, and will initially contain an empty 2D array. This template can be used as a starting point for adding real elevation data later.

**Spatial Extent**: Defines the geographic area covered by the DEM. For this example, we will focus on the western U.S. with specific latitude and longitude boundaries.

**Resolution**: Determines the level of detail in the DEM. A resolution of 0.036 degrees is chosen here, which corresponds to a spatial resolution of approximately 4 kilometers.

**GeoTIFF**: A file format for storing raster graphics, including DEMs. It includes metadata about the spatial reference and other attributes.

In [1]:
import os
import rasterio
from rasterio.transform import from_origin
import numpy as np

`os` module is for handling file operations. `rasterio` for reading and writing raster data in GeoTIFF format. `numpy` is for handling numerical operations and array manipulations

In [2]:
minx, miny, maxx, maxy = -125, 25, -100, 49
resolution = 0.036
width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)
data = np.zeros((height, width), dtype=np.float32)
output_filename = f"../data/dem/western_us_geotiff_template.tif"
with rasterio.open(
    output_filename,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,  # Single band
    dtype=np.float32,
    crs='EPSG:4326',  # WGS84
    transform=from_origin(minx, maxy, resolution, resolution),
) as dst:
    dst.write(data, 1)


`minx` and `maxx`: Define the western and eastern boundaries.

`miny` and `maxy`: Define the southern and northern boundaries.

`resolution`: The distance between adjacent pixels in degrees.

Based on the spatial extent and resolution, we calculate the width and height of the DEM in pixels:

We initialize a NumPy array to hold the DEM data. This array will be empty initially but can be populated with elevation data later

Using the Rasterio library, we create the GeoTIFF file and write the empty data array to it. We specify the metadata, including the coordinate reference system (CRS) and the transformation that maps pixel coordinates to geographic coordinates

## 2. Reprojecting and Resampling DEMs

Below script does 
- Creates a directory for storing template shapefiles.
- Copies the template GeoTIFF into the working directory.
- Generates a shapefile from the template TIFF to use as a clipping mask.
- Reprojects and resamples the DEM using gdalwarp to match the template's spatial extent and resolution.
- Checks the details of the output file to ensure correctness.


```shell
#!/bin/bash
# This script will reproject and resample the western US DEM, clip it, to match the exact spatial extent and resolution as the template TIFF.

cd /home/chetana/gridmet_test_run

mkdir template_shp/

cp /home/chetana/western_us_geotiff_template.tif template_shp/

# Generate the template shape
gdaltindex template.shp template_shp/*.tif

# Reproject and resample the DEM
gdalwarp -s_srs EPSG:4326 -t_srs EPSG:4326 -tr 0.036 0.036 -cutline template.shp -crop_to_cutline -overwrite output_4km.tif output_4km_clipped.tif

# Check the output
gdalinfo output_4km_clipped.tif
```

Generated output looks like this

```Creating output file that is 694P x 666L.
Processing output_4km.tif [1/1] : 0Warning 1: the source raster dataset has a SRS, but the cutline features
not.  We assume that the cutline coordinates are expressed in the destination SRS.
If not, cutline results may be incorrect.
...10...20...30...40...50...60...70...80...90...100 - done.
Driver: GTiff/GeoTIFF
Files: output_4km_clipped.tif
Size is 694, 666
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    USAGE[
        SCOPE["Horizontal component of 3D system."],
        AREA["World."],
        BBOX[-90,-180,90,180]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-125.000000000000000,49.000000000000000)
Pixel Size = (0.036000000000000,-0.036000000000000)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (-125.0000000,  49.0000000) (125d 0' 0.00"W, 49d 0' 0.00"N)
Lower Left  (-125.0000000,  25.0240000) (125d 0' 0.00"W, 25d 1'26.40"N)
Upper Right (-100.0160000,  49.0000000) (100d 0'57.60"W, 49d 0' 0.00"N)
Lower Right (-100.0160000,  25.0240000) (100d 0'57.60"W, 25d 1'26.40"N)
Center      (-112.5080000,  37.0120000) (112d30'28.80"W, 37d 0'43.20"N)
Band 1 Block=694x2 Type=Float32, ColorInterp=Gray
```

## 3. Calculating DEM Features

Extract various terrain features from the DEM, such as slope, aspect, curvature, northness, and eastness, and save the results.

In [3]:
import numpy as np
import pandas as pd
from osgeo import gdal
import warnings
import rasterio
import csv
from rasterio.transform import Affine
from scipy.ndimage import sobel, gaussian_filter

In [4]:
def lat_lon_to_pixel(lat, lon, geotransform):
    x = int((lon - geotransform[0]) / geotransform[1])
    y = int((lat - geotransform[3]) / geotransform[5])
    return x, y

Converts latitude and longitude coordinates to pixel coordinates using the geotransform of the raster. The geotransform provides the mapping between geographic coordinates and pixel locations.

In [5]:
def calculate_slope_aspect(dem_file):
    with rasterio.open(dem_file) as dataset:
        dem_data = dataset.read(1)
        transform = dataset.transform
        dx, dy = np.gradient(dem_data)
        slope = np.arctan(np.sqrt(dx**2 + dy**2))
        slope = 90 - np.degrees(slope)
        aspect = np.degrees(np.arctan2(-dy, dx))
        aspect[aspect < 0] += 360
    return slope, aspect


Read DEM Data: Open the DEM file and read its data.

Calculate Gradients: Use np.gradient to compute the gradient in the x (dx) and y (dy) directions.

Calculate Slope: Use the gradients to compute slope using the arctangent function.

Calculate Aspect: Compute the direction of the slope (aspect) using the arctangent function.

Adjust Aspect Values: Ensure aspect values are between 0 and 360 degrees.

In [6]:
def calculate_curvature(dem_file, sigma=1):
    with rasterio.open(dem_file) as dataset:
        dem_data = dataset.read(1)
        dx = sobel(dem_data, axis=1, mode='constant')
        dy = sobel(dem_data, axis=0, mode='constant')
        dxx = sobel(dx, axis=1, mode='constant')
        dyy = sobel(dy, axis=0, mode='constant')
        curvature = dxx + dyy
        curvature = gaussian_filter(curvature, sigma)
    return curvature


Read DEM Data: Open the DEM file and read its data.

Calculate Gradients: Use Sobel filters to calculate gradients in the x and y directions.

Calculate Curvature: Compute the second derivatives (dxx and dyy) and sum them to get curvature. Apply Gaussian smoothing to reduce noise.

In [7]:
def calculate_gradients(dem_file):
    with rasterio.open(dem_file) as dataset:
        dem_data = dataset.read(1)
        dy, dx = np.gradient(dem_data, dataset.res[0], dataset.res[1])
        northness = np.arctan(dy / np.sqrt(dx**2 + dy**2))
        eastness = np.arctan(dx / np.sqrt(dx**2 + dy**2))
    return northness, eastness


Read DEM Data: Open the DEM file and read its data.

Calculate Gradients: Compute gradients in the y (dy) and x (dx) directions.

Calculate Northness: Compute the northness (gradient in the north direction).

Calculate Eastness: Compute the eastness (gradient in the east direction).


In [8]:
def geotiff_to_csv(geotiff_file, csv_file, column_name):
    with rasterio.open(geotiff_file) as dataset:
        data = dataset.read(1)
        transform = dataset.transform
        height, width = data.shape
        with open(csv_file, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Latitude', 'Longitude', 'x', 'y', column_name])
            for y in range(height):
                for x in range(width):
                    image_value = data[y, x]
                    lon, lat = transform * (x, y)
                    csvwriter.writerow([lat, lon, x, y, image_value])


Read GeoTIFF Data: Open the GeoTIFF file and read its data.

Transform Data: Convert pixel coordinates to geographic coordinates using the geotransform.

Write CSV File: Save the data to a CSV file with columns for latitude, longitude, pixel coordinates, and the data value.


In [10]:
def save_as_geotiff(data, output_file, src_file):
    """
    Save data as a GeoTIFF file with metadata from the source file.

    Args:
        data (array): Data to be saved.
        output_file (str): Path to the output GeoTIFF file.
        src_file (str): Path to the source GeoTIFF file to inherit metadata from.
    """
    with rasterio.open(src_file) as src_dataset:
        profile = src_dataset.profile
        transform = src_dataset.transform

        # Update the data type, count, and set the transform for the new dataset
        profile.update(dtype=rasterio.float32, count=1, transform=transform)

        # Create the new GeoTIFF file
        with rasterio.open(output_file, 'w', **profile) as dst_dataset:
            # Write the data to the new GeoTIFF
            dst_dataset.write(data, 1)

In [11]:
result_dem_csv_path = "../data/dem/dem_template.csv"
result_dem_feature_csv_path = "../data/dem/dem_all.csv"

dem_file = "../data/dem/dem_file.tif"
slope_file = '../data/dem/dem_file.tif_slope.tif'
aspect_file = '../data/dem/dem_file.tif_aspect.tif'
curvature_file = '../data/dem/curvature_file.tif'
northness_file = '../data/dem/northness_file.tif'
eastness_file = '../data/dem/eastness_file.tif'

slope, aspect = calculate_slope_aspect(dem_file)
# slope = calculate_slope(dem_file)
# aspect = calculate_aspect(dem_file)
curvature = calculate_curvature(dem_file)
northness, eastness = calculate_gradients(dem_file)

# Save the slope and aspect as new GeoTIFF files
save_as_geotiff(slope, slope_file, dem_file)
save_as_geotiff(aspect, aspect_file, dem_file)
save_as_geotiff(curvature, curvature_file, dem_file)
save_as_geotiff(northness, northness_file, dem_file)
save_as_geotiff(eastness, eastness_file, dem_file)

geotiff_to_csv(dem_file, dem_file+".csv", "Elevation")
geotiff_to_csv(slope_file, slope_file+".csv", "Slope")
geotiff_to_csv(aspect_file, aspect_file+".csv", "Aspect")
geotiff_to_csv(curvature_file, curvature_file+".csv", "Curvature")
geotiff_to_csv(northness_file, northness_file+".csv", "Northness")
geotiff_to_csv(eastness_file, eastness_file+".csv", "Eastness")

# List of file paths for the CSV files
csv_files = [dem_file+".csv", slope_file+".csv", aspect_file+".csv", 
                curvature_file+".csv", northness_file+".csv", eastness_file+".csv"]

# Initialize an empty list to store all dataframes
dfs = []

# Read each CSV file into separate dataframes
for file in csv_files:
    df = pd.read_csv(file, encoding='utf-8')
    dfs.append(df)

# Merge the dataframes based on the latitude and longitude columns
merged_df = dfs[0]  # Start with the first dataframe
for i in range(1, len(dfs)):
    merged_df = pd.merge(merged_df, dfs[i], on=['Latitude', 'Longitude', 'x', 'y'])

# check the statistics of the columns
for column in merged_df.columns:
    merged_df[column] = pd.to_numeric(merged_df[column], errors='coerce')
    print(merged_df[column].describe())

# Save the merged dataframe to a new CSV file
merged_df.to_csv(result_dem_feature_csv_path, index=False)
print(f"New dem features are updated in {result_dem_feature_csv_path}")

/var/folders/tg/bv4s19f94fv3gqmxhbtgwf_80000gn/T/ipykernel_39509/2570874998.py:5: RuntimeWarning: invalid value encountered in divide
  northness = np.arctan(dy / np.sqrt(dx**2 + dy**2))
/var/folders/tg/bv4s19f94fv3gqmxhbtgwf_80000gn/T/ipykernel_39509/2570874998.py:6: RuntimeWarning: invalid value encountered in divide
  eastness = np.arctan(dx / np.sqrt(dx**2 + dy**2))


count    462204.000000
mean         37.030000
std           6.921275
min          25.060000
25%          31.036000
50%          37.030000
75%          43.024000
max          49.000000
Name: Latitude, dtype: float64
count    462204.00000
mean       -112.52600
std           7.21226
min        -125.00000
25%        -118.77200
50%        -112.52600
75%        -106.28000
max        -100.05200
Name: Longitude, dtype: float64
count    462204.000000
mean        346.500000
std         200.340552
min           0.000000
25%         173.000000
50%         346.500000
75%         520.000000
max         693.000000
Name: x, dtype: float64
count    462204.000000
mean        332.500000
std         192.257631
min           0.000000
25%         166.000000
50%         332.500000
75%         499.000000
max         665.000000
Name: y, dtype: float64
count    462204.000000
mean       1025.629083
std         808.945819
min         -82.938410
25%         180.727250
50%        1018.458080
75%        1604.870600
